In [91]:
import matplotlib.pyplot as plt
%matplotlib inline

In [92]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [93]:
import numpy as np
from scipy.stats import kendalltau
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestRegressor

In [94]:
def read_memfile(filename, shape, dtype='float32'):
    # read binary data and return as a numpy array
    fp = np.memmap(filename, dtype=dtype, mode='r', shape=shape)
    data = np.zeros(shape=shape, dtype=dtype)
    data[:] = fp[:]
    del fp
    return data

In [95]:
def write_memfile(data, filename):
    # write a numpy array 'data' into a binary  data file specified by
    # 'filename'
    shape = data.shape
    dtype = data.dtype
    fp = np.memmap(filename, dtype=dtype, mode='w+', shape=shape)
    fp[:] = data[:]
    del fp

In [96]:
ROOT_DIR = '/rap/jvb-000-aa/COURS2019/etudiants/data/omsignal/myHeartProject/'
TRAIN_LABELED_FILE = 'MILA_TrainLabeledData.dat'
VALIDATION_LABELED_FILE =  'MILA_ValidationLabeledData.dat'

TRAIN_LABELED_FILE = 'MILA_TrainLabeledData.dat'
VALIDATION_LABELED_FILE = 'MILA_ValidationLabeledData.dat'
LABELS_NAME = ["PR_Mean", "RT_Mean", "RR_StdDev", "ID"]

train_labeled_data_file = os.path.join(ROOT_DIR, TRAIN_LABELED_FILE)
validation_labeled_data_file = os.path.join(ROOT_DIR, VALIDATION_LABELED_FILE)

In [97]:
datatrain = read_memfile(train_labeled_data_file, shape=(160,3754))
train_data, train_labels = datatrain[:,:-4], datatrain[:,-4:]

In [98]:
datavalid = read_memfile(validation_labeled_data_file, shape=(160,3754))
valid_data, valid_labels = datavalid[:,:-4], datavalid[:,-4:]

In [99]:
class Preprocessor(nn.Module):

    def __init__(
            self,
            ma_window_size=2,
            mv_window_size=4,
            num_samples_per_second=125):
        # ma_window_size: (in seconds) window size to use
        #                 for moving average baseline wander removal
        # mv_window_size: (in seconds) window size to use
        #                 for moving average RMS normalization

        super(Preprocessor, self).__init__()

        # Kernel size to use for moving average baseline wander removal: 2
        # seconds * 125 HZ sampling rate, + 1 to make it odd

        self.maKernelSize = (ma_window_size * num_samples_per_second) + 1

        # Kernel size to use for moving average normalization: 4
        # seconds * 125 HZ sampling rate , + 1 to make it odd

        self.mvKernelSize = (mv_window_size * num_samples_per_second) + 1

    def forward(self, x):

        with torch.no_grad():
            x = x.unsqueeze(0)
            

            # Remove window mean and standard deviation

            x = (x - torch.mean(x, dim=2, keepdim=True)) / \
                (torch.std(x, dim=2, keepdim=True) + 0.00001)

            # Moving average baseline wander removal

            x = x - F.avg_pool1d(
                x, kernel_size=self.maKernelSize,
                stride=1, padding=(self.maKernelSize - 1) // 2
            )

            # Moving RMS normalization

            x = x / (
                torch.sqrt(
                    F.avg_pool1d(
                        torch.pow(x, 2),
                        kernel_size=self.mvKernelSize,
                        stride=1, padding=(self.mvKernelSize - 1) // 2
                    )) + 0.00001
            )

        # Don't backpropagate further

        x = x.detach().contiguous()

        return x.squeeze(0)

In [100]:
def Plot_ECG(data, start=0, end=30, index_nb=[0], seconde=True):
    """
    Plot the ECG on a user defined time interval
    
    Parameters
    ----------
    data : numpy array
        The ECG Data
    start: int
        The starting time of the part of the ECG you want to plot
    end: int
        The End time of the part of the ECG you want to plot
    index_nb: List
        The index of the observation you want to plot
    seconde: boolean
        Used True if the start time is in seconde
    """
    # Adjust the shape in order to have 2 dimension
    if len(data.shape) == 1:
        data = np.expand_dims(data, axis=0)
        
    # Transform the start and end time 
    if seconde == True:
        start = start*125
        end = end*125
        
    x_labels = np.arange(start, end)
             
    for i, ex in enumerate(index_nb):
        plt.figure(i*2+1) 
        plt.plot(x_labels, data[ex, start:end])
        
    plt.show

In [101]:
class id_trasformer:
    def __init__(self):
        self.DictId2Class = {}
        self.DictClass2Id = {}
        self.n_Id = 0 
        
    def create_dict_ID(self, ID_vector):
        for id in ID_vector:
            if id not in self.DictId2Class:
                self.DictId2Class[id] = self.n_Id
                self.DictClass2Id[self.n_Id] = id
                self.n_Id += 1
                
    def Id2Class(self, ID_vector):
        return [self.DictId2Class[id] for id in ID_vector]
    
    def Class2Id(self, class_vector):
        return [self.DictClass2Id[labels] for labels in class_vector]

In [102]:
device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")

In [103]:
preprocessor = Preprocessor().to(device)

In [104]:
train = torch.tensor(train_data).to(device)
valid =  torch.tensor(valid_data).to(device)
preprocess_train = preprocessor(train)
preprocess_valid = preprocessor(valid)

In [105]:
idtransformer = id_trasformer()
idtransformer.create_dict_ID(train_labels[:,-1])
train_labels[:,-1] = idtransformer.Id2Class(train_labels[:,-1])
valid_labels[:,-1] = idtransformer.Id2Class(valid_labels[:,-1])

Algo to detect R P,T peak and onset offset based on https://pure.tugraz.at/ws/portalfiles/portal/1312717/Online%20and%20Offline%20Determination%20of%20QT%20PR%20in%20Electrocardiography%20LNCS%207719.pdfdf.pdf

In [106]:
def detect_R_peak(data, SADA_wd_size = 7, FS_wd_size = 12, Threshold = 35):
    """
    Take a Batch of ECG data and find the location of the R Peak
    
    The algorithm is based on the paper:
    Online and Offline Determination of QT and PR Interval and QRS Duration in Electrocardiography
    (Bachler et al., 2012)
    The variable name and default value follow the paper
    
    Parameters
    ----------
    data : numpy array
        The ECG Data (batch size x lenght of the ECG recording)
    SADA_wd_size: int
        size of the moving window used in the calculation of SA and DA
    FS_wd_size: int
        size of the moving window used in the calculation of the feature signal FS
    Threshold: int
        FS is compared to the Threshold to determined if its a QRS zone. 
    """
    
    R_peak = []
    
    #Allow batch size of 1
    if len(data.size()) == 1:
        data = data.unsqueeze(0)
    
    D = data[:, 1:] - data[:, 0:-1]
    
    
    data = data.unsqueeze(0)
    D = D.unsqueeze(0)
    SA = F.max_pool1d(data, kernel_size = SADA_wd_size, stride = 1)
    SA = SA + F.max_pool1d(-data, kernel_size = SADA_wd_size, stride = 1) 
    DA = F.max_pool1d(D, kernel_size = SADA_wd_size, stride = 1, padding=1)
    DA = DA + F.max_pool1d(-D, kernel_size = SADA_wd_size, stride = 1, padding=1) 
    
    C = DA[:,:,1:] * torch.pow(SA, 2)
    FS = F.max_pool1d(C, kernel_size = FS_wd_size, stride = 1) 
    Detect = (FS > Threshold)
    
    Detect = Detect.squeeze(0).cpu()
    data = data.squeeze(0).cpu()

    for ECG in range(len(data)):
        
        in_QRS = 0
        start_QRS = 0
        end_QRS = 0
        r_peak = np.array([])
        
        for tick, detect in enumerate(Detect[ECG]):
            
            if (in_QRS == 0) and (detect == 1):
                start_QRS = tick
                in_QRS = 1
                
            elif (in_QRS == 1) and (detect == 0):
                end_QRS = tick
                R_tick = torch.argmax(data[ECG, start_QRS : end_QRS+SADA_wd_size+FS_wd_size]).item()
                r_peak = np.append(r_peak, R_tick + start_QRS)
                in_QRS = 0
                start_QRS = 0
                
        R_peak.append(r_peak)
        
    return R_peak

In [107]:
R_peak_train = detect_R_peak(preprocess_train)
R_peak_valid = detect_R_peak(preprocess_valid)

In [108]:
def Get_RR_Mean_std(R_peak, MaxInterval=180):
    """
    Calculate the mean RR interval and the std
    
    Parameters
    ----------
    R_peak : list of list
        Each entry is a list of the positiion of the R peak in the ECG
    MaxInterval: int
        maximum lenght of an interval, interval higher than this amount are ignore
    """
    #calculate the lenght of the interval
    RR_interval = [R_peak[i][1:]-R_peak[i][0:-1] for i in range(len(R_peak))]
    
    #We keep only good quality one
    RR_interval_adj = [interval[interval<180] for interval in RR_interval]
    

    RR_interval_mean = [np.mean(interval) for interval in RR_interval_adj]    
    RR_std = [np.std(interval) for interval in RR_interval_adj]
    
    return RR_interval_mean, RR_std

In [109]:
Train_RR_mean, Train_RR_std = Get_RR_Mean_std(R_peak_train)
Valid_RR_mean, Valid_RR_std = Get_RR_Mean_std(R_peak_valid)

In [110]:
def create_template(data, R_peak, template_size = 110, All_window = True):
    
    listoftemplate1 = []
    listoftemplate2 = []
    half_size_int = template_size//2
    listECG_id = []
    
    for recording in range(len(data)):
        template = []
        ECG_id = []
        #generate the template      
        
        for i in R_peak[recording][1:-1]:
            new_heart_beat = data[recording][int(i)-int(half_size_int*0.8): int(i)+int(half_size_int*1.2)]
            if len(new_heart_beat) != 110:
                print("error")
            template.append(new_heart_beat)
            
            if All_window:
                ECG_id.append(recording)        

        if All_window == False:
            template = np.mean(template, axis = 0)
            template = np.expand_dims(template, axis = 0)
            ECG_id.append(recording)  
            
        listoftemplate1 = listoftemplate1 + template
        listECG_id = listECG_id + ECG_id
        
        
    #listoftemplate = [hearth_beat for hearth_beat in template for template in listoftemplate]
    return np.array(listoftemplate1), np.array(listECG_id)

In [111]:
def create_template_average(data, R_peak, template_size = 110):
    half_size_int = template_size//2
    listoftemplate = []
    for recording in range(len(R_peak)):
        #generate the template
        template = np.zeros((1,int(half_size_int*0.8)+int(half_size_int*1.2)))
        for i in R_peak[recording][1:-1]:
            new_heart_beat = data[recording][int(i)-int(half_size_int*0.8): int(i)+int(half_size_int*1.2)]
            template = np.concatenate((template,
                                       np.expand_dims(new_heart_beat, axis = 0))
                                       , axis=0)


        template = np.mean(template, axis = 0)
        listoftemplate.append(template)
        
        
        
    #listoftemplate = [hearth_beat for hearth_beat in template for template in listoftemplate]
    return listoftemplate

Here we should probably combine the two function (create_template,create_template_average) above into one

In [112]:
template_valid, t_valid_labels = create_template(preprocess_valid.cpu().numpy(),  R_peak_valid)
template_train, t_train_labels = create_template(preprocess_train.cpu().numpy(), R_peak_train)

In [113]:
Avg_template_valid = create_template_average(preprocess_valid.cpu().numpy(),  R_peak_valid)
Avg_template_train = create_template_average(preprocess_train.cpu().numpy(), R_peak_train)
Avg_template_train = np.array(Avg_template_train)
Avg_template_valid = np.array(Avg_template_valid)

Convolution for classification on all window

In [114]:
import torch
from torchvision import models,transforms,datasets
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline 
import torch.optim as optim
from PIL import Image
import copy
import numpy as np
import torchvision
import os

In [115]:
dictECG_id_to_labels1 = {i:j for i, j in enumerate(train_labels[:,-1])}
dictECG_id_to_labels2 = {i:j for i, j in enumerate(valid_labels[:,-1])}
labels_train = np.array([dictECG_id_to_labels1[i] for i in t_train_labels])
labels_valid = np.array([dictECG_id_to_labels2[i] for i in t_valid_labels])

In [116]:
data_train = torch.Tensor(template_train)
labels_train_t = torch.LongTensor(labels_train)
trainloader = torch.utils.data.TensorDataset(data_train, labels_train_t)
loader_train = torch.utils.data.DataLoader(trainloader, batch_size=128, shuffle = True)

data_valid = torch.Tensor(template_valid)
labels_valid_t = torch.LongTensor(labels_valid)
validloader = torch.utils.data.TensorDataset(data_valid, labels_valid_t)
loader_valid = torch.utils.data.DataLoader(validloader, batch_size=128, shuffle = False)

In [117]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=nn.Conv1d(1,10,kernel_size=5) #22
        self.conv2=nn.Conv1d(10,20,kernel_size=5) #18
        self.drop_conv2=nn.Dropout2d(p=0.5) #4
        self.fc1=nn.Linear(60,50)
        self.drop_lin=nn.Dropout(p=0.5)
        self.fc2=nn.Linear(50,32)
    
    def forward(self,x):
        x = x.unsqueeze(1)
        x=F.relu(F.max_pool1d(self.conv1(x),5))
        x=F.relu(F.max_pool1d(self.drop_conv2(self.conv2(x)),5))
        x=x.view(-1,60)
        x=F.relu(self.drop_lin(self.fc1(x)))
        x=F.log_softmax(self.fc2(x), dim = 1)
    
        return x

In [118]:
model = Net().to(device)

In [119]:
def train(num_epochs, model, lr = 0.1):
  
    optmizer=optim.SGD(model.parameters(),lr=lr)
    best_model=best_model=copy.deepcopy(model)
    best_accuracy=0
    best_epoch=0
  
    for i in range(num_epochs):

        for mode in ["train","eval"]:

            accuracy=0
            loss_total=0

            if mode == "train":

                model.train()
                for data, labels in loader_train:
                    data, labels=data.to(device), labels.to(device)
                    optmizer.zero_grad()

                    output=model(data)
                    pred=torch.argmax(output,1)
                    loss=F.nll_loss(output, labels)

                    accuracy+=(pred==labels).sum()
                    loss_total+=loss

                    loss.backward()
                    optmizer.step()

                if i % 25 == 0:
                    print(f"Epoch{i} Training loss: {loss_total.item()}")
                    print(f"Training Accuracy:{accuracy.item()/len(loader_train.dataset)}")

            if mode == "eval":

                model.eval()
                for data, labels in loader_valid:
                    data, labels=data.to(device), labels.to(device)

                    with torch.no_grad():
                        optmizer.zero_grad()

                        output=model(data)
                        pred=torch.argmax(output,1)
                        loss=F.nll_loss(output, labels)

                        accuracy+=(pred==labels).sum()
                        loss_total+=loss

                accuracy=accuracy.item()/len(loader_valid.dataset)

                if i % 25 == 0:
                    print(f"Eval loss: {loss_total.item()}")
                    print(f"Eval Accuracy:{accuracy}")

                if accuracy > best_accuracy:
                    best_accuracy=accuracy
                    best_model=copy.deepcopy(model)
                    best_epoch=i

    print(f"The best epoch:{best_epoch} with an accuracy of {best_accuracy}")

    return best_model

In [120]:
best_model = train(3000, model,0.1 )


Epoch0 Training loss: 138.79293823242188
Training Accuracy:0.03467980295566502
Eval loss: 137.66673278808594
Eval Accuracy:0.03402041224734841
Epoch25 Training loss: 62.71392059326172
Training Accuracy:0.483743842364532
Eval loss: 113.14208984375
Eval Accuracy:0.4006403842305383
Epoch50 Training loss: 48.044490814208984
Training Accuracy:0.6041379310344828
Eval loss: 114.03598022460938
Eval Accuracy:0.45267160296177705
Epoch75 Training loss: 43.2799186706543
Training Accuracy:0.6364532019704433
Eval loss: 115.51734161376953
Eval Accuracy:0.5157094256553932
Epoch100 Training loss: 41.2182502746582
Training Accuracy:0.6510344827586206
Eval loss: 108.62220001220703
Eval Accuracy:0.5229137482489493
Epoch125 Training loss: 40.48299789428711
Training Accuracy:0.6711330049261084
Eval loss: 104.4974594116211
Eval Accuracy:0.42525515309185513
Epoch150 Training loss: 34.27692794799805
Training Accuracy:0.7152709359605911
Eval loss: 100.2510757446289
Eval Accuracy:0.5295177106263759
Epoch175 Trai

Epoch1450 Training loss: 22.867382049560547
Training Accuracy:0.8086699507389162
Eval loss: 108.82743072509766
Eval Accuracy:0.5941564938963378
Epoch1475 Training loss: 22.27448081970215
Training Accuracy:0.8199014778325123
Eval loss: 126.30577087402344
Eval Accuracy:0.5711426856113668
Epoch1500 Training loss: 22.885955810546875
Training Accuracy:0.81064039408867
Eval loss: 111.17328643798828
Eval Accuracy:0.5935561336802081
Epoch1525 Training loss: 22.35572052001953
Training Accuracy:0.8124137931034483
Eval loss: 107.14436340332031
Eval Accuracy:0.6169701821092656
Epoch1550 Training loss: 23.127561569213867
Training Accuracy:0.8074876847290641
Eval loss: 110.12303924560547
Eval Accuracy:0.6163698218931358
Epoch1575 Training loss: 22.603683471679688
Training Accuracy:0.8175369458128079
Eval loss: 102.8823471069336
Eval Accuracy:0.6245747448469081
Epoch1600 Training loss: 23.95785903930664
Training Accuracy:0.8096551724137931
Eval loss: 115.81414031982422
Eval Accuracy:0.601761056633980

Epoch2900 Training loss: 22.710254669189453
Training Accuracy:0.8100492610837439
Eval loss: 100.52517700195312
Eval Accuracy:0.6521913147888734
Epoch2925 Training loss: 26.58837890625
Training Accuracy:0.7917241379310345
Eval loss: 115.9793701171875
Eval Accuracy:0.5437262357414449
Epoch2950 Training loss: 24.181312561035156
Training Accuracy:0.8055172413793104
Eval loss: 117.44644927978516
Eval Accuracy:0.5903542125275165
Epoch2975 Training loss: 21.181730270385742
Training Accuracy:0.8267980295566503
Eval loss: 122.10824584960938
Eval Accuracy:0.6225735441264759
The best epoch:2919 with an accuracy of 0.6778066840104062


Pred and majority vote

In [133]:
y_pred = best_model(data_valid.to(device))

In [134]:
pred = torch.argmax(y_pred, dim=1)

In [135]:
pred = pred.cpu().numpy()
y_pred_majority = np.array([])
for i in range(160):
    index_of_int = t_valid_labels == i
    counts = np.bincount(pred[index_of_int].astype(int))
    y_pred_majority = np.append(y_pred_majority,np.argmax(counts))

Evaluate using the TA formula

In [136]:
ecgIdAccuracy = recall_score(valid_labels[:,-1], y_pred_majority, average='macro')
adjustementTerm = 1.0 / len(np.unique(valid_labels[:,-1]))
ecgIdAccuracy = (ecgIdAccuracy - adjustementTerm) / (1 - adjustementTerm)
if ecgIdAccuracy < 0:
    ecgIdAccuracy = 0.0
print(ecgIdAccuracy)

0.7548387096774193


End of Convolution


Using the convolution on the average window instead of all window was tested also
Better result are obtained by using all the window and doing a majority vote

 however they both fell short of a PCA + LDA classificateur

In [137]:
labels_train = train_labels[:,-1]
labels_valid = valid_labels[:,-1]

shuffle1 = np.random.permutation(len(Avg_template_train))
train_shuffle = Avg_template_train[shuffle1]
TrainLabel_shuffle = labels_train[shuffle1]
shuffle2 = np.random.permutation(len(Avg_template_valid))
valid_shuffle = Avg_template_valid[shuffle2]
ValidLabel_shuffle = labels_valid[shuffle2]

In [138]:
from sklearn.decomposition import PCA

In [139]:
ipca = PCA(n_components=30)
ipca.fit(train_shuffle)
pca_train = ipca.transform(train_shuffle)
pca_valid = ipca.transform(valid_shuffle)

In [140]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [141]:
lda = LinearDiscriminantAnalysis()
lda.fit(pca_train, TrainLabel_shuffle)
y_pred = lda.predict(pca_valid)
lda.score(pca_valid, ValidLabel_shuffle)

0.78125

In [142]:
ecgIdAccuracy = recall_score(ValidLabel_shuffle, y_pred, average='macro')
adjustementTerm = 1.0 / len(np.unique(ValidLabel_shuffle))
ecgIdAccuracy = (ecgIdAccuracy - adjustementTerm) / (1 - adjustementTerm)
if ecgIdAccuracy < 0:
    ecgIdAccuracy = 0.0
print(ecgIdAccuracy)

0.7741935483870968


Try adding the hearth beat information it does not add anything..

In [143]:
pca_train = np.concatenate((pca_train,np.expand_dims(np.array(Train_RR_mean), axis = 1)[shuffle1]),axis = 1)
pca_train = np.concatenate((pca_train,np.expand_dims(np.array(Train_RR_std), axis = 1)[shuffle1]),axis = 1)
pca_valid = np.concatenate((pca_valid,np.expand_dims(np.array(Valid_RR_mean), axis = 1)[shuffle2]),axis = 1)
pca_valid = np.concatenate((pca_valid,np.expand_dims(np.array(Valid_RR_std), axis = 1)[shuffle2]),axis = 1)

In [144]:
lda = LinearDiscriminantAnalysis()
lda.fit(pca_train, TrainLabel_shuffle)
y_pred = lda.predict(pca_valid)
lda.score(pca_valid, ValidLabel_shuffle)

0.78125

No improvement from adding the hearth beat

Also Try (code not keep): individually analyse each window with PCA + LDA and do With Majority vote 
dont gives better than analyse the average window. 

Regression Task on averaged window with multitask learning

In [145]:
labels_train = train_labels[:]
labels_valid = valid_labels[:]
data_train = torch.Tensor(Avg_template_train)
labels_train_t = torch.Tensor(labels_train)
trainloader = torch.utils.data.TensorDataset(data_train, labels_train_t)
loader_train = torch.utils.data.DataLoader(trainloader, batch_size=160, shuffle = False)

data_valid = torch.Tensor(Avg_template_valid)
labels_valid_t = torch.Tensor(labels_valid)
validloader = torch.utils.data.TensorDataset(data_valid, labels_valid_t)
loader_valid = torch.utils.data.DataLoader(validloader, batch_size=160, shuffle = False)

In [150]:
class Reg_Net(nn.Module):
    def __init__(self):
        super(Reg_Net,self).__init__()
        self.conv1=nn.Conv1d(1,10,kernel_size=3) #22
        self.conv2=nn.Conv1d(10,20,kernel_size=3) #18
        self.conv3=nn.Conv1d(20,20,kernel_size=3) #18
        self.drop_conv3=nn.Dropout2d(p=0.5) #4

        
        self.fc1RR_std=nn.Linear(60,30)
        self.fc2RR_std=nn.Linear(30,1)
        self.fc1TR_mean=nn.Linear(60,30)
        self.fc2TR_mean=nn.Linear(30,1)
        self.fc1PR_mean=nn.Linear(60,30)
        self.fc2PR_mean=nn.Linear(30,1)
    
    def forward(self,x):
        x = x.unsqueeze(1)
        x=F.relu(F.max_pool1d(self.conv1(x),3))
        x=F.relu(F.max_pool1d(self.conv2(x),3))
        x=F.relu(F.max_pool1d(self.drop_conv3(self.conv3(x)),3))
        x=x.view(-1,60)
        
        
        xRR_std = F.relu((self.fc1RR_std(x)))
        xRR_std = self.fc2RR_std(xRR_std)

        
        
        xTR_mean = F.relu((self.fc1TR_mean(x)))
        xTR_mean = self.fc2TR_mean(xTR_mean)

        
        xPR_mean = F.relu((self.fc1PR_mean(x)))
        xPR_mean = self.fc2PR_mean(xPR_mean) 

    
        return xRR_std, xTR_mean, xPR_mean

In [154]:
reg_model = Reg_Net().to(device)

In [155]:
def train(num_epochs, lr=0.1):
  
    optmizer=optim.Adam(reg_model.parameters(),lr=lr)
    reg_best_model =copy.deepcopy(reg_model)
    best_kendall=0
    best_epoch=0
    criterion = nn.MSELoss()
  
    for i in range(num_epochs):

        for mode in ["train","eval"]:

            accuracy=0
            loss_total=0

            if mode == "train":

                reg_model.train()
                for data, labels in loader_train:
                    data, labels=data.to(device), labels.to(device)
                    optmizer.zero_grad()
                    xTR_mean = reg_model(data)

                    xRR_std, xTR_mean, xPR_mean=reg_model(data)
                    LossRR_std = criterion(xRR_std,labels[:,2].unsqueeze(1))
                    LossTR_mean = criterion(xTR_mean,labels[:,1].unsqueeze(1))
                    LossPR_mean = criterion(xPR_mean,labels[:,0].unsqueeze(1))
                    loss = LossRR_std + LossTR_mean + LossPR_mean
                    
                    loss.backward()
                    
                    optmizer.step()

                if i % 1000 == 0:
                    with torch.no_grad():
                        kendallTR_mean = kendalltau(xTR_mean.cpu().detach().numpy(), labels[:,1].cpu().detach().numpy())[0]
                        kendallRR_std = kendalltau(xRR_std.cpu().detach().numpy(), labels[:,2].cpu().detach().numpy())[0]
                        kendallPR_mean = kendalltau(xPR_mean.cpu().detach().numpy(), labels[:,0].cpu().detach().numpy())[0]
                        kendallAvg = np.mean([kendallRR_std, kendallTR_mean, kendallPR_mean])

                        print(f"Epoch{i} Training loss: {loss.item()}")
                        print(f"Training RR STD loss: {LossRR_std.item()}")
                        print(f"Training TR Mean loss: {LossTR_mean.item()}")
                        print(f"Training PR Mean loss: {LossPR_mean.item()}")
                        print(f"Training Kendall TR: {kendallTR_mean}")
                        print(f"Training Kendall std: {kendallRR_std}")
                        print(f"Training Kendall PR: {kendallPR_mean}")
                        print(f"Training Avg Kendall: {kendallAvg}")



            if mode == "eval":

                reg_model.eval()
                for data, labels in loader_valid:
                    data, labels=data.to(device), labels.to(device)

                    with torch.no_grad():
                        optmizer.zero_grad()
                        xTR_mean = reg_model(data)

                        xRR_std, xTR_mean, xPR_mean=reg_model(data)
                        LossRR_std = criterion(xRR_std,labels[:,2].unsqueeze(1))
                        LossTR_mean = criterion(xTR_mean,labels[:,1].unsqueeze(1))
                        LossPR_mean = criterion(xPR_mean,labels[:,0].unsqueeze(1))
                        loss = 0.1*LossRR_std + LossTR_mean + LossPR_mean
                        loss = F.mse_loss(xTR_mean,labels)
                        

                        loss_total+=loss

                if i % 1000 == 0:
                    with torch.no_grad():
                        kendallTR_mean = kendalltau(xTR_mean.cpu().detach().numpy(), labels[:,1].cpu().detach().numpy())[0]
                        kendallRR_std = kendalltau(xRR_std.cpu().detach().numpy(), labels[:,2].cpu().detach().numpy())[0]
                        kendallPR_mean = kendalltau(xPR_mean.cpu().detach().numpy(), labels[:,0].cpu().detach().numpy())[0]
                        kendallAvg = np.mean([kendallRR_std, kendallTR_mean, kendallPR_mean])

                        print(f"Epoch{i} Val loss: {loss.item()}")
                        print(f"Val RR STD loss: {LossRR_std.item()}")
                        print(f"Val TR Mean loss: {LossTR_mean.item()}")
                        print(f"Val PR Mean loss: {LossPR_mean.item()}")
                        print(f"Val Kendall TR: {kendallTR_mean}")
                        print(f"Val Kendall std: {kendallRR_std}")
                        print(f"Val Kendall PR: {kendallPR_mean}")
                        print(f"Val Avg Kendall: {kendallAvg}")
                    



                    if kendallAvg > best_kendall:
                        best_kendall = kendallAvg
                        reg_best_model=copy.deepcopy(reg_model)
                        best_epoch=i

    print(f"The best epoch:{best_epoch} with an avg kendall of {best_kendall}")

    return reg_model

In [156]:
reg_model = train(5000, 0.01)

Epoch0 Training loss: 1539.9134521484375
Training RR STD loss: 44.980445861816406
Training TR Mean loss: 1217.284912109375
Training PR Mean loss: 277.6480407714844
Training Kendall TR: -0.008020129050261433
Training Kendall std: -0.0009433962264150943
Training Kendall PR: 0.046701784875847654
Training Avg Kendall: 0.012579419866390376
Epoch0 Val loss: 440.38385009765625
Val RR STD loss: 45.070228576660156
Val TR Mean loss: 1142.0185546875
Val PR Mean loss: 265.08514404296875
Val Kendall TR: 0.179902502617629
Val Kendall std: 0.15716026585514223
Val Kendall PR: 0.31640195390443143
Val Avg Kendall: 0.21782157412573422
Epoch1000 Training loss: 11.605323791503906
Training RR STD loss: 6.517947196960449
Training TR Mean loss: 2.6349005699157715
Training PR Mean loss: 2.4524760246276855
Training Kendall TR: 0.714892287499774
Training Kendall std: 0.4122641509433962
Training Kendall PR: 0.666719083749475
Training Avg Kendall: 0.5979585073975483
Epoch1000 Val loss: 241.9695281982422
Val RR STD

In [ ]:
y_TR_mean= reg_model(data_valid.to(device)).cpu().detach().squeeze().numpy()

In [ ]:
kendalltau(y_TR_mean, labels_valid[:])

In [ ]:
plt.plot(y_TR_mean, labels_train[:], "ro")

Regression Task Baseline PCA + SVM Regressor

In [ ]:
labels_train = train_labels[:]
labels_valid = valid_labels[:]

shuffle1 = np.random.permutation(len(Avg_template_train))
train_shuffle = Avg_template_train[shuffle1]
TrainLabel_shuffle = labels_train[shuffle1]
shuffle2 = np.random.permutation(len(Avg_template_valid))
valid_shuffle = Avg_template_valid[shuffle2]
ValidLabel_shuffle = labels_valid[shuffle2]

In [ ]:
from sklearn.decomposition import PCA
ipca = PCA(n_components=30)
ipca.fit(train_shuffle)
pca_train = ipca.transform(train_shuffle)
pca_valid = ipca.transform(valid_shuffle)

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
param_grid = [{'C': [1, 5, 10, 15, 100, 1000,10000], 'epsilon': [0.001, 0.0001,0.1,0.5,1,10], "degree" : [3,4]}]
svm_regressor = {}
svm = SVR(gamma="auto")
result = []
for i in range(3):
    clf = GridSearchCV(svm, param_grid, cv=5)
    clf.fit(pca_train, TrainLabel_shuffle[:,i])
    y_pred = clf.predict(pca_valid)
    svm_regressor[LABELS_NAME[i]] = copy.deepcopy(clf)
    print(LABELS_NAME[i])
    result.append(kendalltau(y_pred, ValidLabel_shuffle[:,i])[0])
    print(kendalltau(y_pred, ValidLabel_shuffle[:,i])[0])
print("Avg Kendall corr") 
print(np.mean(result))

Plot R peak

In [ ]:
R_peak_amplitude = [[preprocess_train[i][int(j)] for j in record] for i, record in enumerate(R_peak_train)]

In [ ]:
num_to_look = 10
plt.plot(R_peak_train[num_to_look],R_peak_amplitude[num_to_look],"ro")
Plot_ECG(preprocess_train[num_to_look].cpu(),start=0,end=30)

RR std pred vs real

In [ ]:
plt.plot(Train_RR_std, train_labels[:,-2], "ro")

In [ ]:
kendalltau(Train_RR_std, train_labels[:,-2])[0]

Calculate the position of the other points

In [ ]:
valid_labels

In [ ]:
S_numpy = preprocess_train.cpu().numpy()
R_peak = R_peak_train
RR_interval_mean = Train_RR_mean

Main code below note that its the same thing for the T and P peak (we only pass the reverse list for the P peak) so the code could be improve and cut in function...

In [ ]:
listoftemplate = []
listofQRS_offset = []
listofT_peak = []
listofT_offset = []
listofQRS_onset = []
listofP_peak = []
listofP_onset = []
listofR_Peak = []

for recording in range(len(R_peak)):
    #generate the template
    half_size_int = int(RR_interval_mean[recording]//2)
    template = np.ones((1,int(half_size_int*0.8)+int(half_size_int*1.2)))
    for i in R_peak[recording][1:-1]:
        new_heart_beat = S_numpy[recording][int(i)-int(half_size_int*0.8): int(i)+int(half_size_int*1.2)]
        template = np.concatenate((template,
                                   np.expand_dims(new_heart_beat, axis = 0))
                                   , axis=0)
    
    
    template = np.mean(template, axis = 0)
    
    #find QRS offset
    derivative = template[1:] - template[:-1] 
    TA = np.array([])
    TDA = np.array([])
    for i in range(int(half_size_int*0.8)+1, int(half_size_int*0.8)+25):
        TA = np.append(TA,np.max(template[i:i+4]) - np.min(template[i:i+4])) 
        TDA = np.append(TDA,np.max(derivative[i:i+4]) - np.min(derivative[i:i+4]))
    c1 = 0.1
    c2 = 0.1
    TT = c1 * (np.max(TA) - np.min(TA)) + np.min(TA)
    TD = c2 * (np.max(TDA) - np.min(TDA)) + np.min(TDA)
    
    for i in range(0, len(TA)):
        if TA[i] < TT or TDA[i] < TD:
            QRS_offset = i + int(half_size_int*0.8) + 4
            break
        
    #find T peak
    T_peak = QRS_offset + np.argmax(template[QRS_offset:])
    
    #find T wave offset
    
    k = (template[T_peak] - template[-1])/(T_peak-len(template))
    d = template[-1] - k*len(template)
    g = k*np.arange(0,len(template)) + d
    decision = template - g
    T_peak_offset = np.argmin(decision[T_peak:len(template)]) + T_peak
    
    #reverse the template
    reverse_template = np.flip(template)
    
    #Find the QRS onset
    derivative = reverse_template[1:] - reverse_template[:-1] 
    TA = np.array([])
    TDA = np.array([])
    for i in range(int(half_size_int*1.2)+1, int(half_size_int*1.2)+25):
        TA = np.append(TA,np.max(reverse_template[i:i+4]) - np.min(reverse_template[i:i+4])) 
        TDA = np.append(TDA,np.max(derivative[i:i+4]) - np.min(derivative[i:i+4]))
    c1 = 0.5
    c2 = 0.5
    TT = c1 * (np.max(TA) - np.min(TA)) + np.min(TA)
    TD = c2 * (np.max(TDA) - np.min(TDA)) + np.min(TDA)
    
    for i in range(0, len(TA)):
        if TA[i] < TT or TDA[i] < TD:
            QRS_onset = i + int(half_size_int*1.2) + 4
            break
    
    #Find the P peak
    P_peak = QRS_onset + np.argmax(reverse_template[QRS_onset:])  
    
    #Find the P onset
    
    k = (reverse_template[P_peak] - reverse_template[-1])/(P_peak-len(reverse_template))
    d = reverse_template[-1] - k*len(reverse_template)
    g = k*np.arange(0,len(reverse_template)) + d
    decision = reverse_template - g
    P_onset = np.argmin(decision[P_peak:len(reverse_template)]) + P_peak
    
    
    P_peak = len(template)-P_peak-1
    QRS_onset = len(template)-QRS_onset-1
    P_onset = len(template)-P_onset-1
    
    
    listofR_Peak.append(int(half_size_int*0.8)+1)
    listoftemplate.append(template)
    listofQRS_offset.append(QRS_offset)
    listofT_peak.append(T_peak)
    listofT_offset.append(T_peak_offset)
    listofQRS_onset.append(QRS_onset)
    listofP_peak.append(P_peak)
    listofP_onset.append(P_onset)
    

Calculate RT Mean and PR Mean from the points we have found

In [ ]:
RT_Mean = [T_peak-R_peak+1 for T_peak,R_peak in zip(listofT_peak,listofR_Peak)]
PR_Mean = [R_peak-P_peak for R_peak,P_peak in zip(listofR_Peak,listofP_peak)]

In [ ]:
plt.plot(PR_Mean, train_labels[:,0], "ro")
plt.title("Predicted PR Mean Interval vs True")
plt.show

In [ ]:
kendalltau(PR_Mean, train_labels[:,0])

In [ ]:
kendalltau(RT_Mean, train_labels[:, 1])

In [ ]:
plt.plot(RT_Mean, train_labels[:, 1], "ro")
plt.title("Predicted TR Mean Interval vs True")
plt.show

Visualisation of a window with the points find on it

In [ ]:
index_nb = 53
Plot_ECG(listoftemplate[index_nb],0,len(listoftemplate[index_nb]),seconde=False)
plt.plot(listofR_Peak[index_nb]-1, listoftemplate[index_nb][listofR_Peak[index_nb]-1], "o")
plt.plot(listofQRS_offset[index_nb], listoftemplate[index_nb][listofQRS_offset[index_nb]], "o")
plt.plot(listofT_peak[index_nb], listoftemplate[index_nb][listofT_peak[index_nb]], "o")
plt.plot(listofT_offset[index_nb], listoftemplate[index_nb][listofT_offset[index_nb]], "o")
plt.plot(listofQRS_onset[index_nb], listoftemplate[index_nb][listofQRS_onset[index_nb]], "o")
plt.plot(listofP_peak[index_nb], listoftemplate[index_nb][listofP_peak[index_nb]], "o")
plt.plot(listofP_onset[index_nb], listoftemplate[index_nb][listofP_onset[index_nb]], "o")
plt.legend(["","R","QRS offset", "T", "T offset", "QRS onset","P", "P onset" ])

Visualisation of Labels spread

In [ ]:
def vizualize_label(label_name=LABELS_NAME, type_data=train_labels):
    # Plot histogram of the label to see their distribution
    # Not complete 
    for i, ex in enumerate(label_name):
        plt.figure(i*2+1) #to let the index start at 1
        plt.title(ex)
        plt.hist(train_labels[:,i])
        plt.figure(i*2+2)
        plt.plot(train_labels[:,-1], train_labels[:,i],"ro")
    plt.show()
    
vizualize_label()